# Description

*Created by Freyja Björk Dagbjartsdóttir 6/3/21 for the Accelerate Science Bootcamp by Cambridge Spark*

This notebook describes exploratory data analysis and simple linear regression for physical parameter extraction from Fourier Transfrom AC Voltammetry signals. In FTacV one applies large amplitude voltage on an electrochemical system and measures the response current. The large amplitude voltage induces non-linear response from the electrochemical system. The inputs and outputs look something like this:

<table><tr>
<td> <img src="Data/voltage_vs_time.png" width="250"/> </td>
<td> <img src="Data/current_vs_time.png" width="250"/> </td>
</tr></table>

In this system there is 1 electrochemical reaction and no other reaction.

This simulated response is dependent on several physical properties and operational parameters. I have run 10000 simulations where I vary 10 different physical properties randomly (you will see the distributions later). These physical properties can be seperated into mass transfer properties and charge transfer properties: 

Mass transfer:
* Diffusion coefficients of chemical species 1 and 2 ($D1 and $D2)
* Electrolyte viscosity ($visc)

Charge transfer properties: 

* Formal potential ($Ezero) 

* Rate coefficient ($Ks)

* Charge transfer coefficeint ($alpha)

* System internal resistance ($Ru)

* Electrode/electrolyte internal capacitance ($a0,$a1,$a2)

In theory we might be able to extract all of these physical parameters from a single measurement, making FTacV a very powerful technique. In reality fitting the curves to the data is a slow process and the understanding of the information hidden in the data is scarce as it can be very hard for the human mind to comprehend all that information at once. Pattern recognising ML algorithms are thus a promising way to get a deeper understanding of the technique.  

Since the amount of data is significant and the processing of it takes a very long time, the process will only be described here and not performed. The data folder contains figures and the final set of data that will be visualised and used for simple linear regression.

The way the current signal is usually interpreted is that it is fast Fourier transformed into the frequency domain to obtain the separate harmonics of the signal:

<img src="Data/freq_domain.png" width="250"/>

We pick out the individual harmonics and inverse transfrom them separately into the time domain to obtain an interpretable response. This response typically looks like this:

<table><tr>
<td> <img src="Data/harmonic0.png" width="250"/> </td>
<td> <img src="Data/harmonic1.png" width="250"/> </td>
<td> <img src="Data/harmonic2.png" width="250"/> </td>
</tr></table>
<table><tr>
<td> <img src="Data/harmonic3.png" width="250"/> </td>
<td> <img src="Data/harmonic4.png" width="250"/> </td>
<td> <img src="Data/harmonic5.png" width="250"/> </td>
</tr></table>

We can reduce these data sets without loosing much information to look like the one below. This way the dataset is reduced to 1/16th of the previous one.

<table><tr>
<td> <img src="Data/harmonic0_red.png" width="250"/> </td>
<td> <img src="Data/harmonic1_red.png" width="250"/> </td>
<td> <img src="Data/harmonic2_red.png" width="250"/> </td>
</tr></table>
<table><tr>
<td> <img src="Data/harmonic3_red.png" width="250"/> </td>
<td> <img src="Data/harmonic4_red.png" width="250"/> </td>
<td> <img src="Data/harmonic5_red.png" width="250"/> </td>
</tr></table>

Information about the physical system is embedded in the peak size, shape and location. So instead of throwing the whole time series at a model it might be more beneficial to describe the curves using location and amplitude. This, in principle, is what I do and use in this notebook. Getting this information is a slow process so I will provide the results in  'peaks_for_regression.csv'. There are many ways to present the full data set shown in the first figures to a ML alorithm, but I started with what I think is intuitive for users of this technique. 

**Importing modules**

In [ ]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from bokeh.plotting import Figure, output_notebook, show, save
from bokeh.models.widgets import Tabs, Panel
from bokeh.layouts import row, gridplot
output_notebook()


# Getting the data

The dataframe 'variables' holds our target variables, and peak_info holds descriptions of different peaks amplitudes and location, as well as the value of the minimum current for each harmonic. 

In [ ]:
variables = pd.read_csv('Data/variables.csv',index_col=0)
peak_info = pd.read_csv('Data/peaks_for_regression.csv',header=[0,1,2],index_col=0)
harmonics_number = 5

In [ ]:
peak_info.head(3)

In [ ]:
variables.head(3)

You can modify the following cell to get better sense of what the data looks like.

In [ ]:
harmonic = slice(None)
data_type = 'min_value'
peak_number = slice(None)
peak_info.loc[:,(harmonic,data_type,peak_number)].describe()

# Cleaning and sorting data

The following cell sorts the data depending on if the reaction can be classified as reversible or not. It is common practice to distinguish between reversible or irreversible reactions in electrochemistry.

To aid with visualisations later on, reversible reactions will be labelled *blue* and irreversible reactions will be labelled *red*. 

In [ ]:
#Sort by reversibility:
variables_rev = variables.copy()
variables_rev.insert(0,'Is reversible',None)
reversible_mask = variables_rev['$Ks'] >= 0.1

for i in range(len(reversible_mask)):
    if reversible_mask.iloc[i] == True:
        variables_rev['Is reversible'].iloc[i] = 'blue'
    else:
        variables_rev['Is reversible'].iloc[i] = 'red'

In [ ]:
variables_rev.head(3)

Since the different variables are randomly generated individually, some of the simulations will not be representative of measurements one would do. One would always adjust experimental parameters so that the maximum peaks are clear and visible away from the "edges" of the experiment. The following cell sorts out the bad data in a new dataframe.

In [ ]:
# Remove bad simulations 
peak_info_clean = peak_info.copy()
variables_clean = variables.copy()
variables_rev_clean = variables_rev.copy()
peak_info_clean.replace(511,0,inplace=True)
peak_info_clean.replace(512,0,inplace=True)
peak_info_clean.replace(1023,0,inplace=True)
for i in peak_info_clean.columns:
    if (i[1] == 'location') & (i[0] != str(0)):
        mask = peak_info_clean[i] != 0
        peak_info_clean = peak_info_clean[mask]
        variables_rev_clean = variables_rev_clean[mask]
        variables_clean = variables_clean[mask]
print(peak_info_clean.shape)

Instead of the back peaks I will use some symmetry descriptors for the wave since we know there is information in the symmetry, and it is uncertain if the describing values for the second half of the peaks include much more information than the first half.

In [ ]:
peak_info_clean_sym = peak_info_clean.copy()
#zeroth harmonic 
peak_info_clean_sym[(str(0),'value',str(2))] = (peak_info_clean_sym[(str(0),'value',str(2))] - peak_info_clean[(str(0),'value',str(1))])/peak_info_clean_sym[(str(0),'value',str(1))]
# all other harmonics
for ind in peak_info_clean.columns:
    h = int(ind[0])
    pn = int(ind[2])
    if (pn > h) & (ind[0] != str(0)):
        diff = pn-h
        if diff == 1:
            ind_ref = (ind[0],ind[1],str(pn-1))
        elif diff == 3:
            ind_ref = (ind[0],ind[1],str(pn-3))
        elif diff == 5:
            ind_ref = (ind[0],ind[1],str(pn-5))
        elif diff == 7:
            ind_ref = (ind[0],ind[1],str(pn-7))
        elif diff == 9:
            ind_ref = (ind[0],ind[1],str(pn-9))
        if ind[1] == 'location':
            peak_info_clean_sym[ind] = (peak_info_clean_sym[ind]-511.5)/(511.5-peak_info_clean_sym[ind_ref])
        elif ind[1] == 'value':
            peak_info_clean_sym[ind] = (peak_info_clean_sym[ind] - peak_info_clean[ind_ref])/peak_info_clean_sym[ind_ref]
print(peak_info_clean_sym.shape)

You can modify the following cell to get better sense of what the modfied data looks like.

In [ ]:
harmonic = slice(None)
data_type = 'min_value'
peak_number = slice(None)
peak_info_clean_sym.loc[:,(harmonic,data_type,peak_number)].describe()

# Visualising the data

Whilst there are 68 descriptors I have chosen to use for inital parameter extraction, I will pick 1 or 2 peaks to check visually for correlation with all the target variables.

For the ones where there is 1 maximum peak (harmonics 0,1,3,5) I use the maximum peak but for harmonics 2,4 I use the average of the two center/largest peaks. I will also visualise the relationship between the minimums and target variables. 

In the following figures, you can toggle between different variables, and you will see 6 figures at a time. Each figure represents a harmonic in the following order: [[0,1,2],[3,4,5]]. The red dots represent irreversible reactions, and the blue dots represent reversible reactions.

**Front peak amplitude**

In [ ]:
panels_front_max = []
for var in variables.columns:
    plot_front_max = {}
    if (var == '$D1') | (var == '$D2') | (var == '$Ks') :
        islog = 'log'
    else:
        islog = 'linear'
    n_maxpeak = [1,1,1.5,2,2.5,3]
    for n in range(harmonics_number+1):
        x_vals = variables_rev_clean[var]
        if (n/2 != round(n/2)) | (n == 0) :
            ind = (str(n),'value',str(n_maxpeak[n]))
            y_vals = peak_info_clean_sym[ind]
        else:
            ind1 = (str(n),'value',str(int(n_maxpeak[n]-0.5)))
            ind2 = (str(n),'value',str(int(n_maxpeak[n]+0.5)))
            y_vals = (peak_info_clean_sym[ind1]+peak_info_clean_sym[ind2])/2
        
        plot_front_max[n] = Figure(x_axis_type=islog)
        plot_front_max[n].scatter(x=x_vals, y=y_vals,fill_color=variables_rev_clean['Is reversible'],line_color=None,alpha=0.05)
        plot_front_max[n].toolbar_location=None  
    gridplot_front_max = gridplot([[plot_front_max[0],plot_front_max[1],plot_front_max[2]], [plot_front_max[3],plot_front_max[4],plot_front_max[5]]], plot_width=300, plot_height=200)

    panels_front_max.append(Panel(child=gridplot_front_max, title=var))

full_plot_front_max = Tabs(tabs=panels_front_max)
show(full_plot_front_max)

**Front peak location**

In [ ]:
panels_front_max_loc = []
for var in variables.columns:
    plot_front_max_loc = {}
    if (var == '$D1') | (var == '$D2') | (var == '$Ks') :
        islog = 'log'
    else:
        islog = 'linear'
    n_maxpeak = [1,1,1.5,2,2.5,3]
    for n in range(harmonics_number+1):
        x_vals = variables_rev_clean[var]
        if n == 0:
            x_vals = []
            y_vals = []
        elif (n/2 != round(n/2)) :
            ind = (str(n),'location',str(n_maxpeak[n]))
            y_vals = peak_info_clean_sym[ind]
        else:
            ind1 = (str(n),'location',str(int(n_maxpeak[n]-0.5)))
            ind2 = (str(n),'location',str(int(n_maxpeak[n]+0.5)))
            y_vals = (peak_info_clean_sym[ind1]+peak_info_clean_sym[ind2])/2
        
        plot_front_max_loc[n] = Figure(x_axis_type=islog)
        if n == 0:
            plot_front_max_loc[n].scatter(x=x_vals, y=y_vals,line_color=None,alpha=0.05)
        else:
            plot_front_max_loc[n].scatter(x=x_vals, y=y_vals,fill_color=variables_rev_clean['Is reversible'],line_color=None,alpha=0.05)
        plot_front_max_loc[n].toolbar_location=None  
    gridplot_front_max_loc = gridplot([[plot_front_max_loc[0],plot_front_max_loc[1],plot_front_max_loc[2]], [plot_front_max_loc[3],plot_front_max_loc[4],plot_front_max_loc[5]]], plot_width=300, plot_height=200)

    panels_front_max_loc.append(Panel(child=gridplot_front_max_loc, title=var))

full_plot_front_max_loc = Tabs(tabs=panels_front_max_loc)
show(full_plot_front_max_loc)

**Amplitude symmetry**

In [ ]:
panels_max_sym = []
for var in variables.columns:
    plot_max_sym = {}
    if (var == '$D1') | (var == '$D2') | (var == '$Ks') :
        islog = 'log'
    else:
        islog = 'linear'
    n_maxpeak = [2,2,3.5,5,6.5,8]
    for n in range(harmonics_number+1):
        x_vals = variables_rev_clean[var]
        if (n/2 != round(n/2)) | (n == 0) :
            ind = (str(n),'value',str(n_maxpeak[n]))
            y_vals = peak_info_clean_sym[ind]
        else:
            ind1 = (str(n),'value',str(int(n_maxpeak[n]-0.5)))
            ind2 = (str(n),'value',str(int(n_maxpeak[n]+0.5)))
            y_vals = (peak_info_clean_sym[ind1]+peak_info_clean_sym[ind2])/2
        
        plot_max_sym[n] = Figure(x_axis_type=islog)
        plot_max_sym[n].scatter(x=x_vals, y=y_vals,fill_color=variables_rev_clean['Is reversible'],line_color=None,alpha=0.05)
        plot_max_sym[n].toolbar_location=None  
    gridplot_max_sym = gridplot([[plot_max_sym[0],plot_max_sym[1],plot_max_sym[2]], [plot_max_sym[3],plot_max_sym[4],plot_max_sym[5]]], plot_width=300, plot_height=200)

    panels_max_sym.append(Panel(child=gridplot_max_sym, title=var))

full_plot_max_sym = Tabs(tabs=panels_max_sym)
show(full_plot_max_sym)

**Location symmetry**

In [ ]:
panels_sym_max_loc = []
for var in variables.columns:
    plot_sym_max_loc = {}
    if (var == '$D1') | (var == '$D2') | (var == '$Ks') :
        islog = 'log'
    else:
        islog = 'linear'
    n_maxpeak = [1,1,1.5,2,2.5,3]
    for n in range(harmonics_number+1):
        x_vals = variables_rev_clean[var]
        if n == 0:
            x_vals = []
            y_vals = []
        elif (n/2 != round(n/2)) :
            ind = (str(n),'location',str(n_maxpeak[n]))
            y_vals = peak_info_clean_sym[ind]
        else:
            ind1 = (str(n),'location',str(int(n_maxpeak[n]-0.5)))
            ind2 = (str(n),'location',str(int(n_maxpeak[n]+0.5)))
            y_vals = (peak_info_clean_sym[ind1]+peak_info_clean_sym[ind2])/2
        
        plot_sym_max_loc[n] = Figure(x_axis_type=islog)
        if n == 0:
            plot_sym_max_loc[n].scatter(x=x_vals, y=y_vals,line_color=None,alpha=0.05)
        else:
            plot_sym_max_loc[n].scatter(x=x_vals, y=y_vals,fill_color=variables_rev_clean['Is reversible'],line_color=None,alpha=0.05)
        plot_sym_max_loc[n].toolbar_location=None  
    gridplot_sym_max_loc = gridplot([[plot_sym_max_loc[0],plot_sym_max_loc[1],plot_sym_max_loc[2]], [plot_sym_max_loc[3],plot_sym_max_loc[4],plot_sym_max_loc[5]]], plot_width=300, plot_height=200)

    panels_sym_max_loc.append(Panel(child=gridplot_sym_max_loc, title=var))

full_plot_sym_max_loc = Tabs(tabs=panels_sym_max_loc)
show(full_plot_sym_max_loc)

**Minimum values**

In [ ]:
panels_min = []
for var in variables.columns:
    plot_min = {}
    if (var == '$D1') | (var == '$D2') | (var == '$Ks') :
        islog = 'log'
    else:
        islog = 'linear'
    for n in range(harmonics_number+1):
        x_vals = variables_rev_clean[var]
        ind = (str(n),'min_value',str(0))
        y_vals = peak_info_clean_sym[ind]
        
        plot_min[n] = Figure(x_axis_type=islog)
        plot_min[n].scatter(x=x_vals, y=y_vals,fill_color=variables_rev_clean['Is reversible'],line_color=None,alpha=0.05)
        plot_min[n].toolbar_location=None  
    gridplot_min = gridplot([[plot_min[0],plot_min[1],plot_min[2]], [plot_min[3],plot_min[4],plot_min[5]]], plot_width=300, plot_height=200)

    panels_min.append(Panel(child=gridplot_min, title=var))

full_plot_min = Tabs(tabs=panels_min)
show(full_plot_min)

# Linear Regression

In total we have 6 datasets that we can investigate individually. We have the full set, that we can split into reversible and irreversible sets. We also have data sets only with peak amplitudes and locations, and sets with symmetry informatin. 

The first thing to do is to split these different datasets up into train and test sets. 




In [ ]:
train_ratio = 0.9
test_ratio = 0.1

#Full data set 
X_train, X_test, y_train, y_test = train_test_split(peak_info_clean, variables_clean, test_size=test_ratio, random_state=1)
#Full data set - symmetry descriptors
X_sym_train, X_sym_test, y_sym_train, y_sym_test = train_test_split(peak_info_clean_sym, variables_clean, test_size=test_ratio, random_state=1)
#Reversible data set 
X_rev_train, X_rev_test, y_rev_train, y_rev_test = train_test_split(peak_info_clean[reversible_mask], variables_clean[reversible_mask], test_size=test_ratio, random_state=1)
#Reversible data set - symmetry descriptors
X_rev_sym_train, X_rev_sym_test, y_rev_sym_train, y_rev_sym_test = train_test_split(peak_info_clean_sym[reversible_mask], variables_clean[reversible_mask], test_size=test_ratio, random_state=1)
#Irreversible data set 
X_irrev_train, X_irrev_test, y_irrev_train, y_irrev_test = train_test_split(peak_info_clean[~reversible_mask], variables_clean[~reversible_mask], test_size=test_ratio, random_state=1)
#Irreversible data set - symmetry descriptors
X_irrev_sym_train, X_irrev_sym_test, y_irrev_sym_train, y_irrev_sym_test = train_test_split(peak_info_clean_sym[~reversible_mask], variables_clean[~reversible_mask], test_size=test_ratio, random_state=1)

training_sets = [X_train,X_sym_train,X_rev_train,X_rev_sym_train,X_irrev_train,X_irrev_sym_train]
test_sets = [X_test,X_sym_test,X_rev_test,X_rev_sym_test,X_irrev_test,X_irrev_sym_test]
training_targets = [y_train,y_sym_train,y_rev_train,y_rev_sym_train,y_irrev_train,y_irrev_sym_train]
test_targets = [y_test,y_sym_test,y_rev_test,y_rev_sym_test,y_irrev_test,y_irrev_sym_test]

print(f'Full data set: Train: {X_train.shape[0]} Test:{X_test.shape[0]}')
print(f'Reversible data set: Train: {X_rev_train.shape[0]} Test:{X_rev_test.shape[0]}')
print(f'Irreversible data set: Train: {X_irrev_train.shape[0]} Test:{X_irrev_test.shape[0]}')

Then I create dataframes that can store the R2 values and biases for each linear regression model.

In [ ]:
indices = [['Not known','Not known','Reversible','Reversible','Irreversible','Irreversible'],
          ['Peak values only','Symmetry','Peak values only','Symmetry','Peak values only','Symmetry']]

indices = pd.MultiIndex.from_tuples(list(zip(*indices)), names=["Reaction type", "Data type"])
model_R2 = pd.DataFrame(index = indices,columns = variables.columns)

In [ ]:
model_weights_vars = []

for i in range(len(variables.columns)):
    var = variables.columns[i]
    model_weights_vars.append(pd.DataFrame(index = indices,columns = peak_info.columns))

The following cell trains a separate linear regression model for all datasets and all target variables individually. Each data set is first standardised, followed by training and then testing on a scaled test set. For the target variables that span multiple orders of magnitude, the target sets are transformed to a logarithmic scale. 

In [ ]:
for i in range(len(training_sets)):
    k=0
    for var in variables.columns:
        model = LinearRegression()
        scaler = preprocessing.StandardScaler()
        names = training_sets[i].columns
        training_data = scaler.fit_transform(training_sets[i])
        training_data = pd.DataFrame(training_data, columns=names)
        test_data = scaler.transform(test_sets[i])
        test_data = pd.DataFrame(test_data, columns=names)
        if (var == '$D1') | (var == '$D2') | (var == '$Ks') :
            training_target = np.log(training_targets[i][var])
            test_target = np.log(test_targets[i][var])
        else:
            training_target = training_targets[i][var]
            test_target = test_targets[i][var]
        model.fit(training_data,training_target)
        if i/2 == round(i/2):
            ind2 = 'Peak values only'
        else:
            ind2 = 'Symmetry'
        if (i == 0) | (i == 1) :
            ind1 = 'Not known'
        elif (i == 2) | (i == 3) :
            ind1 = 'Reversible'
        else:
            ind1 = 'Irreversible'
        ind = (ind1,ind2)
        model_R2[var].loc[ind] = model.score(test_data,test_target)
        model_weights_vars[k].iloc[i] = model.coef_
        k = k+1

The 'model_R2' dataframe shows how well a simple linear regression model can extract the physical parameters from the reduced data signal. The 'model_weights_vars' data frames include all weights for the different models, so it can be used to explore the importance of different features. 

In [ ]:
model_R2

The model_weights_vars dataframes include information that can be used to investigate feature importance. 

In [ ]:
i = 2
print(variables.columns[i])
model_weights_vars[i]